In [40]:
from outscraper import ApiClient
import numpy as np
import json
import pandas as pd
import plotly.express as px

In [18]:
# THIS CODE WAS USED TO SCRAPE THE GOOGLE REVIEWS OF SCHOOLS

client = ApiClient(api_key='SECRET_KEY')


sd = open('schools.json')
data = json.load(sd)

context = {'school': [], 'score': [], 'data': []}

for attributes in data['features']:

    school_name = attributes['attributes']['SCHOOL_NAME']
    longitude = attributes['geometry']['x']
    latitude = attributes['geometry']['y']
    results = client.google_maps_reviews(f'{school_name} brampton', reviews_limit=20, limit=1, language='en')

    #creating the context for saving data onto excel spreadsheet

    # adding the info for each review onto the excel spreadsheet
    reviews = results[0]['reviews_data']
    
    for review in reviews:
        context['longitude'] = longitude
        context['latitude'] = latitude
        context['school'].append(school_name)
        context['score'].append(review['review_rating'])
        context['data'].append(review['review_text'])
        
    
data = pd.DataFrame(context)
data.to_excel("review_information.xlsx", index=False)

In [50]:
# THIS WAS USED TO REMOVE CERTAIN CHARACTERS FROM TEXT (THAT PYTHON COULDNT READ)

import urllib
import re

data = pd.read_excel("review_information.xlsx")
data.dropna(inplace=True)
emotions = []

for i in data['data']:
    
    
    reviewtext = re.sub("[^A-Z ]", '', i ,0,re.IGNORECASE)
    reviewtext = reviewtext.encode("ascii", "ignore")
    reviewtext = reviewtext.decode()
    reviewtext = reviewtext.replace(" ", "%20")
    
    url = f"https://nocodefunctions.com/api/sentimentForAText?text-lang=en&text={reviewtext}&explanation=off&output-format=json&explanation-lang=en-US"
    URLresponse = urllib.request.urlopen(url)
    feelingdata = json.loads(URLresponse.read())
    
    if (feelingdata['sentiment'] == "positive feeling"):
        emotions.append(3)
    elif (feelingdata['sentiment'] == "neutral feeling"):
        emotions.append(2)
    else:
        emotions.append(1)

#     print(emotions)

[2]
[2, 2]
[2, 2, 1]
[2, 2, 1, 3]
[2, 2, 1, 3, 2]
[2, 2, 1, 3, 2, 3]
[2, 2, 1, 3, 2, 3, 3]
[2, 2, 1, 3, 2, 3, 3, 3]
[2, 2, 1, 3, 2, 3, 3, 3, 3]


KeyboardInterrupt: 

In [51]:
import pandas as pd
import urllib
import re

data = pd.read_excel("review_information.xlsx")
data.dropna(inplace=True)
emotions = []

for i in data['data']:
    
    reviewtext = re.sub("[^A-Z ]", '', i ,0,re.IGNORECASE)
    reviewtext = reviewtext.encode("ascii", "ignore")
    reviewtext = reviewtext.decode()
    reviewtext = reviewtext.replace(" ", "%20")
    
    url = f"https://nocodefunctions.com/api/sentimentForAText?text-lang=en&text={reviewtext}&explanation=off&output-format=json&explanation-lang=en-US"
    URLresponse = urllib.request.urlopen(url)
    feelingdata = json.loads(URLresponse.read())
    
    if (feelingdata['sentiment'] == "positive feeling"):
        emotions.append(3)
    elif (feelingdata['sentiment'] == "neutral feeling"):
        emotions.append(2)
    else:
        emotions.append(1)

data = pd.read_excel("review_information.xlsx")
data.dropna(inplace=True)
data['opinion'] = emotions

data.to_excel("review_information.xlsx", index=False)

In [97]:
data = pd.read_excel("review_information.xlsx")

for i in range(data.shape[0]):
    if data.loc[i, 'score'] > 3 and data.loc[i, 'opinion'] == 1:
        data.drop(index=i, inplace=True)
    elif (data.loc[i, 'score'] < 2 or data.loc[i, 'score'] > 4) and (data.loc[i, 'opinion'] == 2):
        data.drop(index=i, inplace=True)
    elif data.loc[i, 'score'] < 3 and data.loc[i, 'opinion'] == 3:
        data.drop(index=i, inplace=True)
        
        
data.to_excel("review_information.xlsx", index=False)

In [31]:
# used to find the counter

counter = 1
tally = data.loc[0, 'score']
name = data.loc[0, 'school']

avg_school_ratings = {}

for i in range(1, data.shape[0]):
    
    # if the selected school name is equal to the one being iterated through
    if name == data.loc[i, 'school']:
        tally += data.loc[i, 'score'] # add to the total sum of the school location
        counter += 1 # increment coutner
    
    # if collecting the tallies for the school is complete
    else:
        avg_score = tally / counter # find the average
        avg_school_ratings[name] = avg_score # saving the average ratings for the old school name
        name = data.loc[i, 'school'] # replacing the school name
        tally = 0 # resetting tally
        counter = 1 # resetting counter

avg_score = tally / counter # find the average
avg_school_ratings[name] = avg_score 
    

In [32]:
avg_school_ratings

{'BERYL FORD P.S.': 4.142857142857143,
 'XYNA INTERNATIONAL HIGH SCHOOL': 4.235294117647059,
 'LOTE TREE FOUNDATION': 0.5,
 'LEARNING GENIUS ACADEMY INC': 3.75,
 'KHALSA COMMUNITY SCHOOL': 3.066666666666667,
 'OUR LADY OF LOURDES CATHOLIC E.S.': 2.0,
 'ALGOMA UNIVERSITY': 2.6,
 'MAKKI ISLAMIC LEARNING CENTRE': 3.857142857142857,
 'ST. PATRICK SEPARATE E.S.': 3.25,
 'HOLY NAME OF MARY CATHOLIC S.S.': 1.8,
 'LARKSPUR P.S.': 3.6,
 'ST. FRANCIS XAVIER E.S.': 2.5,
 'ST. URSULA E.S.': 3.8,
 "ST. MARGUERITE D'YOUVILLE S.S.": 3.5555555555555554,
 'PARKHOLME SCHOOL': 2.75,
 "FLETCHER'S MEADOW S.S.": 2.6,
 'ROBERT H. LAGERQUIST SR.P.S.': 2.8,
 'ESKER LAKE P.S.': 3.5714285714285716,
 'FOLKSTONE P.S.': 3.375,
 'ST. AGNES E.S.': 0.0,
 'ST. KEVIN SEPARATE E.S.': 3.0,
 'ST. JOHN BOSCO E.S.': 3.0,
 'HILLDALE P.S.': 3.0,
 'DORSET DRIVE P.S.': 3.8461538461538463,
 'SPRINGDALE P.S.': 3.7,
 'NORTH PARK S.S.': 3.6363636363636362,
 'SHAW P.S.': 1.5,
 'CARDINAL AMBROZIC CATHOLIC S.S.': 3.125,
 'ST. CECILIA E

In [52]:
# we want to obtain the information to add to the final array sheet
# need the school names, latitude, longitude, avg_rating

display_data = {'school': [], 'latitude': [], 'longitude': [], 'avg_rating': []}

sd = open('schools.json')
filex = json.load(sd)


for i in range(data.shape[0]):
    if data.loc[i, 'school'] not in display_data['school']:
        display_data['school'].append(data.loc[i, 'school'])
        
        for attributes in filex['features']:
            if attributes['attributes']['SCHOOL_NAME'] == data.loc[i, 'school']:
                display_data['latitude'].append(attributes['geometry']['y'])
                display_data['longitude'].append(attributes['geometry']['x'])
            
        display_data['avg_rating'].append(avg_school_ratings[data.loc[i, 'school']])
    

display_data = pd.DataFrame(display_data)
display_data.to_excel("final.xlsx", index=False)


,school,latitude,longitude,avg_rating
0,BERYL FORD P.S.,43.790683,-79.669568,4.142857
1,XYNA INTERNATIONAL HIGH SCHOOL,43.704101,-79.786456,4.235294
2,LOTE TREE FOUNDATION,43.737570,-79.769513,0.500000
3,LEARNING GENIUS ACADEMY INC,43.772437,-79.662223,3.750000
4,KHALSA COMMUNITY SCHOOL,43.730643,-79.740001,3.066667
...,...,...,...,...
227,MY LEARNING OASIS,43.786797,-79.733364,4.166667
228,POLKADOTS PLAYLAND & MONTESSORI CENTRE,43.659979,-79.813178,4.350000
229,QUARK WEST SCHOOL,43.632667,-79.784334,0.000000
230,BONI MONTESSORI,43.749442,-79.728107,3.583333
